In [1]:
# HIDDEN

from datascience import *
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
%matplotlib inline

### Tables

Tables are a fundamental object type for representing data sets. A table can be viewed in two ways. Tables are a sequence of named columns that each describe a single aspect of all entries in a data set. Tables are also a sequence of rows that each contain all information about a single entry in a data set. 

Tables are typically created from files that contain comma-separated values, called CSV files. The file below contains "Annual Estimates of the Resident Population by Single Year of Age and Sex for the United States." 

In [2]:
census_url = 'http://www.census.gov/popest/data/national/asrh/2014/files/NC-EST2014-AGESEX-RES.csv'
full_table = Table.read_table(census_url)
full_table

SEX,AGE,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014
0,0,3944153,3944160,3951330,3963071,3926665,3945610,3948350
0,1,3978070,3978090,3957888,3966510,3978006,3943077,3962123
0,2,4096929,4096939,4090862,3971573,3979952,3992690,3957772
0,3,4119040,4119051,4111920,4102501,3983049,3992425,4005190
0,4,4063170,4063186,4077552,4122303,4112638,3994047,4003448
0,5,4056858,4056872,4064653,4087713,4132210,4123408,4004858
0,6,4066381,4066412,4073013,4074979,4097780,4143094,4134352
0,7,4030579,4030594,4043047,4083240,4084964,4108615,4154000
0,8,4046486,4046497,4025604,4053206,4093213,4095827,4119524
0,9,4148353,4148369,4125415,4035769,4063193,4104133,4106832


A [description of the table](http://www.census.gov/popest/data/national/asrh/2014/files/NC-EST2014-AGESEX-RES.pdf) appears online. The `SEX` column contains numeric codes: `0` stands for the total, `1` for male, and `2` for female. The `AGE` column contains ages, but the special value `999` is a sum of the total population. The rest of the columns contain estimates of the US population.

Typically, a public table will contain more information than necessary for a particular investigation or analysis. In this case, let us suppose that we are only interested in the population changes from 2010 to 2014. We can select only a subset of the columns using the `select` method.

In [3]:
census = full_table.select(['SEX', 'AGE', 'POPESTIMATE2010', 'POPESTIMATE2014'])
census

SEX,AGE,POPESTIMATE2010,POPESTIMATE2014
0,0,3951330,3948350
0,1,3957888,3962123
0,2,4090862,3957772
0,3,4111920,4005190
0,4,4077552,4003448
0,5,4064653,4004858
0,6,4073013,4134352
0,7,4043047,4154000
0,8,4025604,4119524
0,9,4125415,4106832


The names used for columns in the original table can be changed, often to clarify or simplify future computations. Be careful, the `relabel` method does alter the table, so that the original name is lost.

In [4]:
census.relabel('POPESTIMATE2010', '2010')
census.relabel('POPESTIMATE2014', '2014')
census

SEX,AGE,2010,2014
0,0,3951330,3948350
0,1,3957888,3962123
0,2,4090862,3957772
0,3,4111920,4005190
0,4,4077552,4003448
0,5,4064653,4004858
0,6,4073013,4134352
0,7,4043047,4154000
0,8,4025604,4119524
0,9,4125415,4106832


Each column of a table is an array of the same length, and so columns can be combined. Columns are accessed and assigned by name using square brackets.

In [5]:
census['Change'] = census['2014'] - census['2010']
census['Growth'] = census['Change'] / census['2010']
census

SEX,AGE,2010,2014,Change,Growth
0,0,3951330,3948350,-2980,-0.000754176
0,1,3957888,3962123,4235,0.00107002
0,2,4090862,3957772,-133090,-0.0325335
0,3,4111920,4005190,-106730,-0.0259562
0,4,4077552,4003448,-74104,-0.0181736
0,5,4064653,4004858,-59795,-0.014711
0,6,4073013,4134352,61339,0.0150599
0,7,4043047,4154000,110953,0.0274429
0,8,4025604,4119524,93920,0.0233307
0,9,4125415,4106832,-18583,-0.00450452


Although the columns of this table are simply arrays of numbers, the format of those numbers can be changed to improve the interpretability of the table. The `set_format` method takes `Formatter` objects, which exist for dates (`DateFormatter`), currencies (`CurrencyFormatter`), numbers, and percentages.

In [6]:
census.set_format('Growth', PercentFormatter)
census.set_format(['2010', '2014', 'Change'], NumberFormatter)
census

SEX,AGE,2010,2014,Change,Growth
0,0,"3,951,330","3,948,350","-2,980",-0.08%
0,1,"3,957,888","3,962,123","4,235",0.11%
0,2,"4,090,862","3,957,772","-133,090",-3.25%
0,3,"4,111,920","4,005,190","-106,730",-2.60%
0,4,"4,077,552","4,003,448","-74,104",-1.82%
0,5,"4,064,653","4,004,858","-59,795",-1.47%
0,6,"4,073,013","4,134,352","61,339",1.51%
0,7,"4,043,047","4,154,000","110,953",2.74%
0,8,"4,025,604","4,119,524","93,920",2.33%
0,9,"4,125,415","4,106,832","-18,583",-0.45%


The information in a table can be accessed in many ways. The attributes `column_labels`, `columns`, and `rows` demonstrated below are used for any table.

In [7]:
census.column_labels

('SEX', 'AGE', '2010', '2014', 'Change', 'Growth')

In [8]:
len(census)

6

In [9]:
len(census.rows)

306

In [10]:
census.rows[0]

Row(SEX=0, AGE=0, _2=3951330, _3=3948350, Change=-2980, Growth=-0.00075417644185628633)

In [11]:
census.columns[2]

array([  3951330,   3957888,   4090862,   4111920,   4077552,   4064653,
         4073013,   4043047,   4025604,   4125415,   4187063,   4115511,
         4113280,   4119666,   4145614,   4231003,   4313252,   4376368,
         4491005,   4571385,   4568470,   4387914,   4286987,   4217221,
         4243589,   4289424,   4160810,   4237010,   4247540,   4210285,
         4304244,   4042507,   3967603,   3933570,   3822188,   3948318,
         3830202,   3896776,   4080229,   4324460,   4387483,   4163470,
         4082733,   4093841,   4178501,   4438551,   4529712,   4535464,
         4534669,   4599102,   4646251,   4498994,   4480630,   4439422,
         4288475,   4259012,   4093161,   3946548,   3802467,   3694276,
         3616733,   3520121,   3495073,   3652174,   2706063,   2678532,
         2621346,   2693709,   2359819,   2167833,   2062581,   1953614,
         1883821,   1750309,   1685999,   1631879,   1481683,   1449173,
         1402184,   1354913,   1319725,   1212603, 

In [12]:
census.rows[0][2]

3951330

In [13]:
census.columns[2][0]

3951330

Let's take a look at the growth rates of the total number of males and females by selecting only the *rows* that sum over all ages. This sum is expressed with the special value `999` according to this data set description.

In [14]:
census.where('AGE', 999)

SEX,AGE,2010,2014,Change,Growth
0,999,"309,347,057","318,857,056","9,509,999",3.07%
1,999,"152,089,484","156,936,487","4,847,003",3.19%
2,999,"157,257,573","161,920,569","4,662,996",2.97%


What ages of males are driving this rapid growth? We can first filter the `census` table to keep only the male entries, then sort by growth rate in decreasing order.

In [15]:
males = census.where('SEX', 1)
males.sort('Growth', descending=True)

SEX,AGE,2010,2014,Change,Growth
1,99,"6,104","9,037","2,933",48.05%
1,100,"9,351","13,729","4,378",46.82%
1,98,"9,504","13,649","4,145",43.61%
1,93,"60,182","85,980","25,798",42.87%
1,96,"22,022","31,235","9,213",41.84%
1,94,"43,828","62,130","18,302",41.76%
1,97,"14,775","20,479","5,704",38.61%
1,95,"31,736","42,824","11,088",34.94%
1,91,"104,291","138,080","33,789",32.40%
1,92,"83,462","109,873","26,411",31.64%


The fact that there are more men with `AGE` of 100 than 99 looks suspicious; shouldn't there be fewer? A careful look at the description of the data set reveals that the 100 category actually includes all men who are 100 or older. The growth rates in men at these very old ages could have several explanations, such as a large influx from another country, but the most natural explanation is that people are simply living longer in 2014 than 2010.

The `where` method can also take an array of boolean values, constructed by applying some comparison operator to a column of the table. For example, we can find all of the age groups among both sexes for which the absolute `Change` is substantial. The `show` method displays all rows without abbreviating.

In [16]:
both = census.where(census['SEX'] != 0)
both.where(both['Change'] > 200000).sort('AGE').show()

SEX,AGE,2010,2014,Change,Growth
1,23,"2,151,095","2,399,883","248,788",11.57%
2,23,"2,066,126","2,298,701","232,575",11.26%
1,24,"2,161,380","2,391,398","230,018",10.64%
2,24,"2,082,209","2,301,237","219,028",10.52%
1,34,"1,908,761","2,192,455","283,694",14.86%
2,34,"1,913,427","2,170,440","257,013",13.43%
1,57,"1,910,028","2,110,149","200,121",10.48%
2,57,"2,036,520","2,237,030","200,510",9.85%
2,58,"1,963,764","2,163,798","200,034",10.19%
1,59,"1,779,504","2,006,900","227,396",12.78%


Many of the same ages appear for both males (`1`) and females (`2`), and most are clumped together in the 57-67 range. In 2014, these people would be born between 1947 and 1957, the height of the post-WWII baby boom in the United States.

It is possible to specify multiple conditions using the functions `np.logical_and` and `np.logical_or`. When two conditions are combined with `logical_and`, both must be true for a row to be retained. When conditions are combined with `logical_or`, then either one of them must be true for a row to be retained. Here are two different ways to select 18 and 19 year olds.

In [17]:
both.where(np.logical_or(both['AGE']==18, both['AGE']==19))

SEX,AGE,2010,2014,Change,Growth
1,18,"2,305,733","2,165,062","-140,671",-6.10%
1,19,"2,334,906","2,220,790","-114,116",-4.89%
2,18,"2,185,272","2,060,528","-124,744",-5.71%
2,19,"2,236,479","2,105,604","-130,875",-5.85%


In [18]:
both.where(np.logical_and(both['AGE']>=18, both['AGE']<=19))

SEX,AGE,2010,2014,Change,Growth
1,18,"2,305,733","2,165,062","-140,671",-6.10%
1,19,"2,334,906","2,220,790","-114,116",-4.89%
2,18,"2,185,272","2,060,528","-124,744",-5.71%
2,19,"2,236,479","2,105,604","-130,875",-5.85%


Here, we observe a rather dramatic decrease in the number of 18 and 19 year olds in the United States; the children of the baby boom are now even older.

**Aggregation and Grouping.** This particular dataset includes entries for sums across all ages and sexes, using the special values `999` and `0` respectively. However, if these rows did not exist, we would be able to aggregate the remaining entries.

In [19]:
no_sums = both.select(['SEX', 'AGE', '2014']).where(both['AGE'] != 999)
females = no_sums.where('SEX', 2).select(['AGE', '2014'])
sum(females['2014'])

161920569

Some columns express categories, such as the sex or age group in the case of the census table. Aggregation can also be performed on every value for a category using the `group` and `groups` methods. The `group` method takes a single column (or column name) as an argument and collects all values associated with each unique value in that column.

In [20]:
no_sums.group('AGE')

AGE,SEX,2014
0,[1 2],[2017857 1930493]
1,[1 2],[2023253 1938870]
2,[1 2],[2022502 1935270]
3,[1 2],[2048618 1956572]
4,[1 2],[2043498 1959950]
5,[1 2],[2043467 1961391]
6,[1 2],[2110328 2024024]
7,[1 2],[2122240 2031760]
8,[1 2],[2105122 2014402]
9,[1 2],[2097272 2009560]


A second argument, the name of a function, can be provided to `group` in order to aggregate the resulting values. For exmaple, the `sum` function can be used to add together the populations for each age. In this result, the `SEX sum` column is meaningless because the values were simply codes for different categories. However, the `2014 sum` column does in fact contain the total number across all sexes for each age category.

In [21]:
no_sums.group('AGE', sum)

AGE,SEX sum,2014 sum
0,3,3948350
1,3,3962123
2,3,3957772
3,3,4005190
4,3,4003448
5,3,4004858
6,3,4134352
7,3,4154000
8,3,4119524
9,3,4106832


**Joining Tables.** There are many situations in data analysis in which two different rows need to be considered together. Two tables can be joined into one, an operation that creates one long row out of two matching rows. These matching rows can be from the same table or different tables.

For example, we might want to estimate which age categories are expected to change significantly in the future. Someone who is 14 years old in 2014 will be 20 years old in 2020. Therefore, one estimate of the number of 20 year olds in 2020 is the number of 14 year olds in 2014. Between the ages of 1 and 50, annual mortality rates are very low (less than 0.5% for men and less than 0.3% for women [[1](http://www.ssa.gov/oact/STATS/table4c6.html)]). Immigration also affects population changes, but for now we will ignore this confounding factor. Let's consider just females in this analysis.

In [22]:
females['AGE+6'] = females['AGE'] + 6
females

AGE,2014,AGE+6
0,"1,930,493",6
1,"1,938,870",7
2,"1,935,270",8
3,"1,956,572",9
4,"1,959,950",10
5,"1,961,391",11
6,"2,024,024",12
7,"2,031,760",13
8,"2,014,402",14
9,"2,009,560",15


In order to relate the age in 2014 to the age in 2020, we will join this table with itself, matching values in the `AGE` column with values in the `AGE in 2020` column.

In [28]:
joined = females.join('AGE', females, 'AGE+6')
joined

AGE,2014,AGE+6,AGE_2,2014_2
6,2024024,12,0,1930493
7,2031760,13,1,1938870
8,2014402,14,2,1935270
9,2009560,15,3,1956572
10,2015380,16,4,1959950
11,2001949,17,5,1961391
12,1993547,18,6,2024024
13,2041159,19,7,2031760
14,2068252,20,8,2014402
15,2035299,21,9,2009560


The resulting table has five columns. The first three are the same as before. The two new colums are the values for `AGE` and `2014` that appeared in a different row, the one in which that `AGE` appeared in the `AGE+6` column. For instance, the first row contains the number of 6 year olds in 2014 and an estimate of the number of 6 year olds in 2020 (who were 0 in 2014). Some relabeling and selecting makes this table more clear.

In [29]:
future = joined.select(['AGE', '2014', '2014_2']).relabel('2014_2', '2020 (est)')
future

AGE,2014,2020 (est)
6,2024024,1930493
7,2031760,1938870
8,2014402,1935270
9,2009560,1956572
10,2015380,1959950
11,2001949,1961391
12,1993547,2024024
13,2041159,2031760
14,2068252,2014402
15,2035299,2009560


Adding a `Change` column and sorting by that change describes some of the major changes in age categories that we can expect in the United States for people under 50. According to this simplistic analysis, there will be substantially more people in their late 30's by 2020.

In [30]:
predictions = future.where(future['AGE'] < 50)
predictions['Change'] = predictions['2020 (est)'] - predictions['2014']
predictions.sort('Change', descending=True)

AGE,2014,2020 (est),Change
40,1940627,2170440,229813
38,1936610,2154232,217622
30,2110672,2301237,190565
37,1995155,2148981,153826
39,1993913,2135416,141503
29,2169563,2298701,129138
35,2046713,2169563,122850
36,2009423,2110672,101249
28,2144666,2244480,99814
41,1977497,2046713,69216
